## Homegate Data Analysis 

In [103]:
# Preliminary
import pandas as pd
import os
import numpy as np
os.chdir('/Users/jindi/Documents/GitHub/pp4r_final_assignments') # change this to your path

In [104]:
# Read file
df = pd.read_csv('data/Homegate_data.csv')
df.head()

,Net rent:,Add'l expenses:,Rent:,Available from:,Type:,No. of rooms:,Number of toilets:,Floor:,Surface living:,Year built:,Listing ID,Object ref.,Address,URL,Room height:,Last refurbishment:,Number of floors:,Floor space:,Land area:,Number of apartments:,Volume:
0,"CHF 3,015.–",CHF 380.–,"CHF 3,395.–",01.11.2023,Apartment,3.5,1.0,6,88 m2,2015.0,4000128038,7841.03.0602,"Pfingstweidstrasse 106b, 8005 Zürich",https://www.homegate.ch/rent/4000128038,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"CHF 3,050.–",CHF 400.–,"CHF 3,450.–",01.12.2023,Apartment,2.5,NaN,1,80 m2,2017.0,3003561923,l7h0a.if0f3,"Gläserenstrasse 14, 8142 Uitikon Waldegg",https://www.homegate.ch/rent/3003561923,2.5 m,NaN,NaN,NaN,NaN,NaN,NaN
2,"CHF 2,445.–",CHF 323.–,"CHF 2,768.–",Immediately,Apartment,4.5,NaN,4,100 m2,NaN,3003561870,cbfj0.v9g10,"Sihlfeldstrasse 89, 8004 Zürich",https://www.homegate.ch/rent/3003561870,NaN,2023.0,NaN,NaN,NaN,NaN,NaN
3,"CHF 3,235.–",CHF 260.–,"CHF 3,495.–",01.11.2023,Apartment,4.0,NaN,2,95 m2,1921.0,3003561634,335.1.1201,"Kreuzstrasse 6, 8802 Kilchberg",https://www.homegate.ch/rent/3003561634,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,On request,01.10.2023,Apartment,4.5,NaN,24,127 m2,NaN,3003561592,299-9.tower55.28b2c967-74a2-4531-8b53-63317787...,"Hagenholzstrasse, 8050 Zürich, 8050 Zürich",https://www.homegate.ch/rent/3003561592,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleaning

In [105]:
df.rename(columns=lambda x: x.rstrip(':'), inplace=True)

In [106]:
# Define the new, more descriptive column names
new_column_names = [
    'Net Rent (CHF)',
    'Additional Expenses (CHF)',
    'Gross Rent (CHF)',
    'Available Date',
    'Property Type',
    'Number of Rooms',
    'Number of Toilets',
    'Floor Level',
    'Living Area (sq. m.)',
    'Year Built',
    'Listing ID',
    'Object Reference',
    'Address',
    'URL Link',
    'Room Height (m.)',
    'Last Refurbishment Year',
    'Number of Floors',
    'Floor Space (sq. m.)',
    'Land Area (sq. m.)',
    'Number of Apartments',
    'Property Volume (cubic m.)',
]

# Assign the new column names to the DataFrame
df.columns = new_column_names


Change Address

In [107]:
# Define a regular expression pattern to extract street, postcode, and city
pattern = r'^(?P<Street>.*?),\s*(?P<Postcode>\d+)\s+(?P<City>.*)$'
address_df = df['Address'].str.extract(pattern)

# Concatenate the extracted address parts with the original DataFrame
df = pd.concat([df, address_df], axis=1)

# Drop the original "Address" column if you no longer need it
df = df.drop(columns=['Address'])

# Fix the tying error in the city name
df['City'] = df['City'].replace("Zürich, 8050 Zürich", "Zürich")


Change Price

In [108]:
# Remove "CHF," the comma, and ".–" and convert to numeric
columns_to_convert = ['Net Rent (CHF)',
    'Additional Expenses (CHF)',
    'Gross Rent (CHF)',]
for column in columns_to_convert:
    df[column] = df[column].replace('On request', np.nan)  # Replace "On request" with NaN
    df[column] = df[column].str.replace('CHF', '')  # Remove "CHF"
    df[column] = df[column].str.replace(',', '')   # Remove commas
    df[column] = df[column].str.replace('.–', '')  # Remove ".–"


# Convert to numeric, treating NaN as float
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')


<ipython-input-108-1bfe648f4a1d>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('.–', '')  # Remove ".–"


Miscellaneous

In [109]:
df["Living Area (sq. m.)"] = df["Living Area (sq. m.)"].str.replace(' m2', '')  # Remove " m2"
columns_to_convert = ['Living Area (sq. m.)',
    'Number of Rooms',
    'Number of Toilets',
    'Floor Level',
    'Year Built']
for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [110]:
df

,Net Rent (CHF),Additional Expenses (CHF),Gross Rent (CHF),Available Date,Property Type,Number of Rooms,Number of Toilets,Floor Level,Living Area (sq. m.),Year Built,Listing ID,Object Reference,URL Link,Room Height (m.),Last Refurbishment Year,Number of Floors,Floor Space (sq. m.),Land Area (sq. m.),Number of Apartments,Property Volume (cubic m.),Street,Postcode,City
0,3015.0,380.0,3395.0,01.11.2023,Apartment,3.5,1.0,6.0,88.0,2015.0,4000128038,7841.03.0602,https://www.homegate.ch/rent/4000128038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pfingstweidstrasse 106b,8005,Zürich
1,3050.0,400.0,3450.0,01.12.2023,Apartment,2.5,NaN,1.0,80.0,2017.0,3003561923,l7h0a.if0f3,https://www.homegate.ch/rent/3003561923,2.5 m,NaN,NaN,NaN,NaN,NaN,NaN,Gläserenstrasse 14,8142,Uitikon Waldegg
2,2445.0,323.0,2768.0,Immediately,Apartment,4.5,NaN,4.0,100.0,NaN,3003561870,cbfj0.v9g10,https://www.homegate.ch/rent/3003561870,NaN,2023.0,NaN,NaN,NaN,NaN,NaN,Sihlfeldstrasse 89,8004,Zürich
3,3235.0,260.0,3495.0,01.11.2023,Apartment,4.0,NaN,2.0,95.0,1921.0,3003561634,335.1.1201,https://www.homegate.ch/rent/3003561634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kreuzstrasse 6,8802,Kilchberg
4,NaN,NaN,NaN,01.10.2023,Apartment,4.5,NaN,24.0,127.0,NaN,3003561592,299-9.tower55.28b2c967-74a2-4531-8b53-63317787...,https://www.homegate.ch/rent/3003561592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hagenholzstrasse,8050,Zürich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,2690.0,100.0,2790.0,By agreement,Apartment,NaN,NaN,NaN,24.0,NaN,3000390342,Zurich.Militärstrasse 24.Studio Apartment Junior,https://www.homegate.ch/rent/3000390342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Militärstrasse 24,8004,Zurich
974,2090.0,77.0,2167.0,By agreement,Apartment,NaN,NaN,NaN,15.0,NaN,3000390341,Zurich.Militärstrasse 24.Studio Apartment Mini,https://www.homegate.ch/rent/3000390341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Militärstrasse 24,8004,Zurich
975,2690.0,100.0,2790.0,By agreement,Apartment,NaN,NaN,NaN,21.0,NaN,3000390340,Zurich.Militärstrasse 24.1 Bedroom Apartment Mini,https://www.homegate.ch/rent/3000390340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Militärstrasse 24,8004,Zurich
976,4090.0,151.0,4241.0,By agreement,Apartment,NaN,NaN,NaN,43.0,NaN,3000390337,Zurich.Militärstrasse 24.2 Bedroom Apartment J...,https://www.homegate.ch/rent/3000390337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Militärstrasse 24,8004,Zurich
